In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1_l2
from imblearn.over_sampling import SMOTE
from collections import Counter

# Pandas ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Veri setini yükle
df = pd.read_excel("C:/Users/koray/PycharmProjects/pythonProject/DATA SETS/yl11_dataset.xlsx")

# Eksik veri kontrolü ve doldurma
df.fillna(df.median(), inplace=True)

# Aykırı değerleri düzeltme fonksiyonları
def outlier_thresholds(dataframe, variable, low_quantile=0.05, up_quantile=0.95):
    quantile_one = dataframe[variable].quantile(low_quantile)
    quantile_three = dataframe[variable].quantile(up_quantile)
    interquantile_range = quantile_three - quantile_one
    up_limit = quantile_three + 1.5 * interquantile_range
    low_limit = quantile_one - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

num_cols = [col for col in df.columns if df[col].dtypes != "O"]
for col in num_cols:
    replace_with_thresholds(df, col)

# SMOTE ile gözlem birimini artırma
X = df.drop(columns=["su"])
y = df["su"]

# SMOTE uygulaması: Her sınıfın örnek sayısını 10000 yapıyoruz
smote = SMOTE(sampling_strategy={0: 10000, 1: 10000, 2: 10000, 3: 10000, 4: 10000, 5:10000, 6:10000}, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Orijinal veri sınıf dağılımı:", Counter(y))
print("SMOTE sonrası sınıf dağılımı:", Counter(y_resampled))

# Yeni veri seti oluşturma
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), 
                          pd.DataFrame(y_resampled, columns=["su"])], axis=1)

# Özellik mühendisliği
df_resampled['klorofil_su_etkileşim'] = df_resampled['klorofil miktarı'] * df_resampled['su']
df_resampled['sıcaklık_su_oranı'] = df_resampled['bitki örtü sıcaklığı'] / (df_resampled['su'] + 1)
df_resampled["Klorofil_x_Sıcaklık"] = df_resampled["klorofil miktarı"] * df_resampled["bitki örtü sıcaklığı"]
df_resampled['log_klorofil'] = np.log(df_resampled['klorofil miktarı'] + 1)
df_resampled['sqrt_sıcaklık'] = np.sqrt(df_resampled['bitki örtü sıcaklığı'])
df_resampled = pd.get_dummies(df_resampled, columns=["su"], prefix='su', dtype=int)

# Bağımsız ve bağımlı değişkenlerin ayrılması
X = df_resampled.drop(columns=['yaprak sap uzunluğu', 'yaprak ağırlığı'])
y = df_resampled[['yaprak sap uzunluğu', 'yaprak ağırlığı']]

# Verinin ölçeklendirilmesi
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# Eğitim ve test verisinin oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# ================================
# Özellik Seçimi (SelectKBest)
# ================================
# f_regression tek hedef için çalıştığından, ilk hedef sütununu kullanıyoruz (yaprak sap uzunluğu)
selector = SelectKBest(score_func=f_regression, k=15)
X_train_selected = selector.fit_transform(X_train, y_train[:, 0])
X_test_selected = selector.transform(X_test)

# ================================
# PCA ile Boyut Azaltma
# ================================
pca = PCA(n_components=0.95)  # %95 varyansı koru
X_train_pca = pca.fit_transform(X_train_selected)
X_test_pca = pca.transform(X_test_selected)

# ================================
# Farklı Modeller ve Hiperparametre Optimizasyonu
# ================================

# 1. RandomForestRegressor (multi-output desteği vardır)
rf_model = RandomForestRegressor(random_state=42)
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf_grid = GridSearchCV(rf_model, rf_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
rf_grid.fit(X_train_pca, y_train)
print("En iyi RandomForest parametreleri:", rf_grid.best_params_)

# 2. XGBoost: MultiOutputRegressor ile sarmalayarak multi-output desteği sağlıyoruz
xgb_model = MultiOutputRegressor(XGBRegressor(random_state=42))
xgb_params = {
    'estimator__n_estimators': [100, 200],
    'estimator__learning_rate': [0.01, 0.1],
    'estimator__max_depth': [3, 5]
}
xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
xgb_grid.fit(X_train_pca, y_train)
print("En iyi XGBoost parametreleri:", xgb_grid.best_params_)

# ================================
# En İyi Modelin Seçilmesi ve Değerlendirilmesi
# ================================
# Burada örneğin XGBoost modelini seçiyoruz
best_model = xgb_grid.best_estimator_

# Test seti üzerinde tahmin yapma
y_pred_scaled = best_model.predict(X_test_pca)

# Tahminleri ters ölçeklendirme
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test)

# Performans metriklerinin hesaplanması
mae = mean_absolute_error(y_test_actual, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
r2 = r2_score(y_test_actual, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")


C:\Users\koray\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\koray\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:783: UserWarning: k=15 is greater than n_features=14. All the features will be returned.
  warnings.warn(


Orijinal veri sınıf dağılımı: Counter({0: 303, 1: 303, 3: 303, 5: 303, 2: 302, 6: 301, 4: 300})
SMOTE sonrası sınıf dağılımı: Counter({0: 10000, 1: 10000, 2: 10000, 3: 10000, 4: 10000, 5: 10000, 6: 10000})
En iyi RandomForest parametreleri: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
En iyi XGBoost parametreleri: {'estimator__learning_rate': 0.1, 'estimator__max_depth': 5, 'estimator__n_estimators': 200}
Mean Absolute Error (MAE): 1.3051942814970425
Root Mean Squared Error (RMSE): 2.5406087854154893
R-squared (R²): 0.7983700578321485
